In [107]:
# Imports
import pandas

In [108]:
# Part 1 Functions

def create_section_list(total_sections:int, section_assignment: str) -> list[int]:
    """This function creates a list of sections from 0 to total sections.

    Args:
        total_sections (int): The number of sections for elves to clean.
        section_assignment (str): The given section assignment by the task input.

    Returns:
        list[int]: Returns a list of sections that elves can be assigned to.
    """

    # Create a total list.
    total_list = list(range(total_sections+1))


    # Convert the section_assignment to two indices.
    section_assignment_start, section_assignment_end = section_assignment.split("-")
    return total_list[int(section_assignment_start):int(section_assignment_end)+1]

def check_sublist_contains(elf_1_assignment:list[int], elf_2_assignment:list[int], mode: str) -> bool:
    """This function gets the smaller of two lists, and checks if it is in the larger of two list.

    Args:
        elf_1_assignment (list[int]): Elf 1's assignments as list of integers.
        elf_2_assignment (list[int]): Elf 2's assignments as list of integers.
        mode (str): The mode of operation - any, or all.

    Raises:
        ValueError: Raised if there is an unrecognised mode assigned to the function.

    Returns:
        bool: Returns true if smaller is in larger list.
    """

    # First, we get the smallest and largest lists, as the largest list won't fit totally in the smallest list.

    smallest_list = min(elf_1_assignment, elf_2_assignment, key=len)
    largest_list = max(elf_1_assignment, elf_2_assignment, key=len)

    match mode:
        case "all":

            # Prevent a bug where two lengths are the same, and smallest and largest list are assigned to the same object.
            if len(elf_1_assignment) == len(elf_2_assignment):
                return elf_1_assignment == elf_2_assignment

            # Return true if smaller list is explicitly in the larger list.
            return all(item in largest_list for item in smallest_list)

        case "any":
            # Return true if any overlaps between the smaller list and the larger list.
            return any(item in elf_1_assignment for item in elf_2_assignment) or any(item in elf_2_assignment for item in elf_1_assignment)

        case _:
            raise ValueError("The selected mode values are either 'any' or 'all'.")




In [110]:
df = pandas.read_table("./input.txt", header=None)

df.rename({0: "SectionPairs"}, axis=1, inplace=True)

# Split the section pairs into two new assignments.
df['Elf1Assignment'], df['Elf2Assignment'] = zip(*df['SectionPairs'].str.split(","))

# Run our create_section_list onto each assignment.
df['Elf1Assignment'] = df.apply(lambda x: create_section_list(99, x['Elf1Assignment']), 
                        axis=1)

df['Elf2Assignment'] = df.apply(lambda x: create_section_list(99, x['Elf2Assignment']), 
                        axis=1)

# Check if any of the assignments overlap.
df['AssignmentFullyOverlaps'] = df.apply(lambda x: check_sublist_contains(x['Elf1Assignment'], x['Elf2Assignment'], "all"), 
                        axis=1)

df['AssignmentPartiallyOverlaps'] = df.apply(lambda x: check_sublist_contains(x['Elf1Assignment'], x['Elf2Assignment'], "any"), 
                        axis=1)

# Count the number of overlaps.

# NOTE: This was fun - first, I converted everythig to string, but I noticed there were some outliers where one elf assignment was simply '9', and was marking as *in* a list of '58, 59, 60'.
# NOTE: This would have benefitted from writing tests.

print("The answer for part 1 is:")
print(df['AssignmentFullyOverlaps'].values.sum())

print("The answer for part 2 is:")
print(df['AssignmentPartiallyOverlaps'].values.sum())


The answer for part 1 is:
494
The answer for part 2 is:
833
